In [ ]:
import json
import geopandas as gpd
import shapely.geometry as sg
import ee
import geemap

# Initialize the Earth Engine library.
ee.Initialize()

# Define the function to extract GeoJSON information
def extract_geojson_info(file_path, feature_index):
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    if feature_index >= len(data['features']):
        raise ValueError("Feature index out of range")
        
    feature = data['features'][feature_index]
    geometry = feature['geometry']
    
    if geometry['type'] == 'Polygon':
        coordinates = geometry['coordinates']
        # Convert the coordinates into a Shapely Polygon object
        polygon = sg.Polygon(coordinates[0])  # Assuming the first list in coordinates contains the exterior ring
        # Create a GeoDataFrame with the polygon geometry
        gdf = gpd.GeoDataFrame({'geometry': [polygon]}, crs='EPSG:4326')
        # Extract the bounding box coordinates
        bounds = gdf.bounds
        min_lon, min_lat, max_lon, max_lat = bounds.minx.values[0], bounds.miny.values[0], bounds.maxx.values[0], bounds.maxy.values[0]
        # Convert the coordinates into an Earth Engine polygon geometry
        ee_polygon = ee.Geometry.Polygon(coordinates)
    else:
        raise ValueError("Unsupported geometry type. Only 'Polygon' is supported.")
    
    return max_lat, min_lat, max_lon, min_lon, ee_polygon

# Example usage:
file_path = 'Split_GeoJson101.geojson'
feature_index = 0
# Change this to the index of the feature you want to extract
max_lat, min_lat, max_lon, min_lon, houseCoords = extract_geojson_info(file_path, feature_index)

# Create a bounding box for the area of interest (around your house).
boundingBox = ee.Geometry.Rectangle([min_lon, min_lat, max_lon, max_lat])

# Load the image collection.
dataset = ee.ImageCollection('USDA/NAIP/DOQQ').filter(ee.Filter.date('2021-01-01', '2023-12-31'))

# Create an image composite or use the first image.
firstImage = dataset.median()

# Clip the image to the bounding box.
clippedImage = firstImage.clip(boundingBox)

# Create a mask from the houseCoords polygon.
mask = ee.Image.constant(1).clip(houseCoords).mask()

# Apply the mask to the clipped image.
blackImage = ee.Image.constant([0, 0, 0, 255]).clip(boundingBox)
maskedImage = clippedImage.updateMask(mask).blend(blackImage.updateMask(mask.Not()))

# Visualization parameters.
trueColorVis = {
    'bands': ['R', 'G', 'B'],
    'min': 0,
    'max': 255,
}

# Initialize a map.
m = geemap.Map()
m.set_center((max_lon + min_lon) / 2, (max_lat + min_lat) / 2, 20)  # Center the map on the bounding box.
m.add_layer(maskedImage, trueColorVis, 'True Color')

print("Max Latitude:", max_lat)
print("Min Latitude:", min_lat)
print("Max Longitude:", max_lon)
print("Min Longitude:", min_lon)
print("Coords:", houseCoords)

# Display the map.
m